In [1]:
import idx2numpy
import seaborn as sns
X_train_3D = idx2numpy.convert_from_file('MNIST_dataset/train-images-idx3-ubyte')
X_train = X_train_3D.flatten().reshape(60000,784)

y_train = idx2numpy.convert_from_file('MNIST_dataset/train-labels-idx1-ubyte')
print(X_train.shape, y_train.shape)
X_test_3D = idx2numpy.convert_from_file('MNIST_dataset/t10k-images-idx3-ubyte')
X_test =  X_test_3D.flatten().reshape(10000,784)

y_test = idx2numpy.convert_from_file('MNIST_dataset/t10k-labels-idx1-ubyte')
print(X_test.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [2]:
import numpy as np
from sklearn.utils import shuffle
X_shuffle,y_shuffle = shuffle(X_train,y_train)
X_train = X_shuffle[0:50000]
y_train = y_shuffle[0:50000]

from sklearn import tree
from sklearn.model_selection import cross_val_predict

#dt_clf = tree.DecisionTreeClassifier(max_depth=20, max_leaf_nodes=300)
dt_clf = tree.DecisionTreeClassifier(max_depth=8)

y_train_pred = cross_val_predict(dt_clf, X_train, y_train, cv=2)
print(dt_clf.fit(X_train, y_train))

print('Number of nodes in the decision tree {}.'.format(dt_clf.tree_.node_count))
print('Number of threshold in the decision tree {}.'.format(len(dt_clf.tree_.threshold)))
print('Number of leaves in the decision tree {}. It will be number of states in sequential FSM.'.format(dt_clf.tree_.n_leaves))

from sklearn.metrics import accuracy_score
print(accuracy_score(y_train, dt_clf.predict(X_train)), accuracy_score(y_test, dt_clf.predict(X_test)))

DecisionTreeClassifier(max_depth=8)
Number of nodes in the decision tree 503.
Number of threshold in the decision tree 503.
Number of leaves in the decision tree 252. It will be number of states in sequential FSM.
0.83086 0.8187


In [3]:
import pickle
with open('MNIST_dataset/MNIST_dataset.pkl', 'wb') as f:
    pickle.dump([X_train, y_train, X_test, y_test], f)

#### Run this cell twice.

In [5]:
%%capture cap --no-stderr
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}{}:if ({} <= {})".format(indent, node, name, int(round(threshold,3)))) 
            recurse(tree_.children_left[node], depth + 1)
            print ("{}{}:else ".format(indent, node, name, int(round(threshold,3))))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{} Label<={};".format(indent, np.argmax(tree_.value[node][0],axis=0)))

    recurse(0, 1)

cols = range(784)
features = ['pixels[{}]'.format(str(i)) for i in cols]
class_names = [str(i) for i in dt_clf.classes_]
tree_to_code(dt_clf, features)

with open('decision_rules_mnist.txt', 'w') as f:
    f.write(cap.stdout)

#### Run this cell twice.

In [7]:
%%capture cap --no-stderr

import pdb
def write_file(data_towrite):
    with open('verilog_file.v', 'a', encoding='utf-8') as file:
        file.writelines(data_towrite)
        file.close()
        
def write_line_file(data_towrite, line_num):
    with open('verilog_file.v', 'r', encoding='utf-8') as file:
        file_content = file.readlines()
        file.close()
    file_content[line_num] = file_content[line_num].replace('\n','') + ' ' + data_towrite + '\n'  
    with open('verilog_file.v', 'w', encoding='utf-8') as file:
        file.writelines(file_content)
        file.close()
        
def search_content_file(word):
    with open('verilog_file.v', 'r') as file:
        file_content = file.readlines()
        for line_num,line in enumerate(file_content):
            if word in line:
                return(line_num)
        return("content doesn't exists.")
    
def update_stateformat_file():
    with open('verilog_file.v', 'r', encoding='utf-8') as file:
        file_content = file.readlines()
        file.close()
    for line_num,line in enumerate(file_content):
        if 'if' in line:
            file_content[line_num] = '    case ' + file_content[line_num].strip() + ' \n'  
    with open('verilog_file.v', 'w', encoding='utf-8') as file:
        file.writelines(file_content)
        file.close()
        
def clear_file():
    with open('verilog_file.v', 'w', encoding='utf-8') as file:
        pass
        file.close()
        
def print_file_content():
    with open('verilog_file.v', 'r', encoding='utf-8') as file:
        file_content = file.read()
        file.close()
    print(file_content)


import itertools
with open('decision_rules_mnist.txt', 'r') as f:
    tree_verilog = f.read()
    f.close()

clear_file()
line_num = 0
else_state = 0
else_nextif_state = 0
curr_line,next_line = itertools.tee(tree_verilog.split('\n'))
next(next_line, None)
for i,j in list(zip(curr_line,next_line)):
    if 'Label' in i:
        pass
        #write_file(f'  begin {i.strip()} state<=0;ml_inference_completed<=1; end \n')
    elif 'else' in i and 'Label' in j:
        else_state = i.strip().split(':')[0]
        line_num = search_content_file(' '+str(else_state)+':if' )
        spaces = 12-len(str(line_num))
        spaces = ' '*spaces
        write_line_file(f'\n{spaces}else:  {j.strip().replace("<","")} state=0;ml_inference_completed=1; clock_counter = clock_counter+1;', line_num)
    elif 'else' in i and 'if' in j:
        else_state = i.strip().split(':')[0]
        else_nextif_state = j.strip().split(':')[0]
        line_num = search_content_file(' '+str(else_state)+':if' )
        spaces = 12-len(str(line_num))
        spaces = ' '*spaces
        write_line_file(f' \n{spaces}else: state={else_nextif_state}; clock_counter = clock_counter+1;', line_num)
    elif 'if' in i and 'Label' in j:
        write_file(f' {i.strip()}:  {j.strip().replace("<","")} state=0;ml_inference_completed<=1; clock_counter = clock_counter+1; ')
    elif 'if' in i and 'if' in j:
        write_file(f' {i.strip()}: clock_counter = clock_counter+1; state={j.strip().split(":")[0]};  \n ')  
update_stateformat_file()
print('def inference_rate(pixels):')
print(' clock_counter=0; state=0; Label=10; ml_inference_completed=0; #assigning dummy default label to 10')
print(' while(Label == 10):')
print('  match state:')
print_file_content()
print(' return (Label,clock_counter)')

with open('mnist_dt_case_fsm.py', 'w') as f:
    f.write(cap.stdout)

In [8]:
# Aligning the spaces
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm = f.readlines()
    f.close()
    
curr_line,next_line = itertools.tee(py_fsm)
next(next_line, None)
for i,j in list(zip(curr_line,next_line)):
    if 'if' in i and 'else' in j:
        if len(i.split('if')[0]) != len(j.split('else')[0]):
            py_fsm[py_fsm.index(j)] = ' '* (len(i.split('if')[0]) - len(j.split('else')[0])) + py_fsm[py_fsm.index(j)]
            

for i in py_fsm:
    if 'case' in i:
        py_fsm[py_fsm.index(i)] = i.split(':if')[0]+':\n'+ ' '*(len(i.split(':if')[0])+1) + 'if' + i.split(':if')[1]
        
with open('mnist_dt_case_fsm.py', 'w') as f:
    f.writelines(py_fsm)
    f.close()        
    
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm_ap = f.read()
    f.close()
    
print(py_fsm_ap)

def inference_rate(pixels):
 clock_counter=0; state=0; Label=10; ml_inference_completed=0; #assigning dummy default label to 10
 while(Label == 10):
  match state:
    case 0:
           if (pixels[350] <= 126): clock_counter = clock_counter+1; state=1; 
           else: state=248; clock_counter = clock_counter+1;
    case 1:
           if (pixels[568] <= 0): clock_counter = clock_counter+1; state=2; 
           else: state=123; clock_counter = clock_counter+1;
    case 2:
           if (pixels[430] <= 0): clock_counter = clock_counter+1; state=3; 
           else: state=66; clock_counter = clock_counter+1;
    case 3:
           if (pixels[405] <= 0): clock_counter = clock_counter+1; state=4; 
           else: state=35; clock_counter = clock_counter+1;
    case 4:
           if (pixels[485] <= 12): clock_counter = clock_counter+1; state=5; 
           else: state=20; clock_counter = clock_counter+1;
    case 5:
           if (pixels[154] <= 0): clock_counter = clock_counter+1; state=6

In [9]:
#Adding default state 
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm = f.readlines()
    f.close()
py_fsm[-2] = ' '*4+'case _:\n'+ ' '*len(' '*4+'case _:') + 'Label=10; state=0;ml_inference_completed=1; clock_counter = clock_counter+1;\n'

with open('mnist_dt_case_fsm.py', 'w') as f:
    f.writelines(py_fsm)
    f.close()        
    
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm_ap = f.read()
    f.close()
    
print(py_fsm_ap)

def inference_rate(pixels):
 clock_counter=0; state=0; Label=10; ml_inference_completed=0; #assigning dummy default label to 10
 while(Label == 10):
  match state:
    case 0:
           if (pixels[350] <= 126): clock_counter = clock_counter+1; state=1; 
           else: state=248; clock_counter = clock_counter+1;
    case 1:
           if (pixels[568] <= 0): clock_counter = clock_counter+1; state=2; 
           else: state=123; clock_counter = clock_counter+1;
    case 2:
           if (pixels[430] <= 0): clock_counter = clock_counter+1; state=3; 
           else: state=66; clock_counter = clock_counter+1;
    case 3:
           if (pixels[405] <= 0): clock_counter = clock_counter+1; state=4; 
           else: state=35; clock_counter = clock_counter+1;
    case 4:
           if (pixels[485] <= 12): clock_counter = clock_counter+1; state=5; 
           else: state=20; clock_counter = clock_counter+1;
    case 5:
           if (pixels[154] <= 0): clock_counter = clock_counter+1; state=6

#### Extracting the state labels and transition labels.

In [18]:
with open('mnist_dt_case_fsm.py', 'r') as f:
    py_fsm = f.readlines()
    f.close()
    
state_labels = []
transition_labels = []
for i in py_fsm:
    if 'case' in i and '_' not in i:
        state_labels.append(int(i.split(':')[0].split(' ')[-1]))
    if 'state=' in i:
        transition_labels.append(int(i.split('state=')[-1].split(';')[0]))

transition_labels = list(set(transition_labels))
print(state_labels == transition_labels)

print(f'Number of state/transition labels = {len(state_labels)}.')
print(state_labels)

True
Number of state/transition labels = 251.
[0, 1, 2, 3, 4, 5, 6, 7, 10, 13, 14, 17, 20, 21, 22, 25, 28, 29, 32, 35, 36, 37, 38, 41, 44, 45, 48, 51, 52, 53, 56, 59, 60, 63, 66, 67, 68, 69, 70, 73, 76, 77, 80, 83, 84, 85, 88, 92, 93, 94, 95, 98, 101, 102, 105, 108, 109, 110, 113, 116, 117, 120, 123, 124, 125, 126, 127, 128, 131, 134, 135, 138, 141, 142, 143, 146, 149, 150, 153, 156, 157, 158, 159, 162, 165, 166, 170, 171, 172, 175, 178, 179, 182, 185, 186, 187, 188, 189, 192, 195, 196, 199, 202, 203, 204, 207, 210, 211, 214, 217, 218, 219, 220, 223, 226, 227, 230, 233, 234, 235, 238, 241, 242, 245, 248, 249, 250, 251, 252, 253, 254, 257, 260, 261, 264, 267, 268, 269, 272, 275, 276, 279, 282, 283, 284, 285, 288, 291, 292, 295, 298, 299, 300, 303, 306, 307, 310, 313, 314, 315, 316, 317, 320, 323, 324, 327, 330, 331, 332, 335, 338, 339, 342, 345, 346, 347, 348, 351, 354, 355, 358, 361, 362, 363, 366, 369, 370, 373, 376, 377, 378, 379, 380, 381, 384, 387, 388, 391, 394, 395, 396, 399, 402

### Run the next experiments in Python version > 3.10 because 'case' statement is only avaiable after this python version. 

### Next, called the function "inference_rate" from python file "mnist_dt_case_fsm.py".

### Remember that the while loop is added in the windows source code to mimic the clock and continuously run the state machine. 

In [10]:
from mnist_dt_case_fsm import inference_rate

In [11]:
import pickle
with open('MNIST_dataset/MNIST_dataset.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(50000, 784) (50000,) (10000, 784) (10000,)


In [12]:
clock_counter = 0
Y_pred = []
for i,sample in enumerate(X_test):
    inf_label, clock = inference_rate(sample)
    Y_pred.append(inf_label)
    clock_counter = clock_counter + clock

In [13]:
print(f'Total clock used for inferencing 10000 samples = {clock_counter}')

Total clock used for inferencing 10000 samples = 79991


In [14]:
#Verify that the accuracy measured after state-machine inference is same as given by sklearn model.
from sklearn.metrics import accuracy_score
print(f'Accuracy using state machine = {accuracy_score(y_test, Y_pred)}')

Accuracy using state machine = 0.8187


In [15]:
#Sequential circuit using finite state machine
#Average clock used per sample
Average_clock_per_sample = clock_counter/len(X_test)

print(Average_clock_per_sample)

7.9991


In [16]:
clock_period = 1/12000000 # 12 MHz
time_per_sample = Average_clock_per_sample *  clock_period * 1000000
print('Inference time used per sample when operate at 12MHz = ',time_per_sample, 'micro sec.' ) 
print('Inference per sec = ', round(1000000/time_per_sample))

Inference time used per sample when operate at 12MHz =  0.6665916666666668 micro sec.
Inference per sec =  1500169
